In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
max_reviews = 5000
star5 = []
star4 = []
star3 = []
star2 = []
star1 = []
url_fruit = ["https://www.11st.co.kr/products/1102013026", "https://www.11st.co.kr/products/1372129464", "https://www.11st.co.kr/products/1418504992"]

driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url_fruit[0])
time.sleep(3)

body = driver.find_element("tag name", "body")

# 페이지 다운 3번
for _ in range(3):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)


# 1~5점
scores = ['span[data-log-body*="\'btn_name\':\'1\'"]', 'span[data-log-body*="\'btn_name\':\'2\'"]', 'span[data-log-body*="\'btn_name\':\'3\'"]', 
         'span[data-log-body*="\'btn_name\':\'4\'"]', 'span[data-log-body*="\'btn_name\':\'5\'"]']

for idx, score in enumerate(scores):
    driver.switch_to.default_content()
    # 리뷰 탭 클릭
    driver.find_element(By.ID, "tabMenuDetail2").click()
    time.sleep(1)
    
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    driver.switch_to.frame("ifrmReview")
    
    # 평점 전체 클릭
    score_dropdown_btn = driver.find_element(By.ID, 'review-star-dropdown')
    score_dropdown_btn.click()
    time.sleep(1)
    
    # '1~5점' 버튼 클릭
    star_button = driver.find_element(By.CSS_SELECTOR, score)
    driver.execute_script("arguments[0].click();", star_button)
    
    seen_count = 0
    
    while seen_count < max_reviews:
        time.sleep(2)
        items = driver.find_elements(By.CSS_SELECTOR, "li.review_list_element")
        
        # 새로 추가된 리뷰만 처리
        new_items = items[seen_count:]
        
        if not new_items:
            print("새 리뷰 없음")
            break
    
        for item in new_items:
            grade = idx + 1
            try:
                review = item.find_element(By.CSS_SELECTOR, ".cont_review_hide.text-expanded").text.strip()
                if review == "":
                    continue
            except:
                continue
    
            try:
                image_buttons = item.find_elements(By.CSS_SELECTOR, "div.c_product_review_thumbnail2 button")
                image_count = len(image_buttons)
                has_image = "Y" if image_count > 0 else "N"
            except:
                image_count = 0
                has_image = "N"
        
            try:
                help_count = item.find_element(By.CSS_SELECTOR, "i.kkukCount").text
            except:
                help_count = "0"
        
            try:
                date = item.find_element(By.CSS_SELECTOR, "p.side span.date").text
            except:
                date = ""
        
            result = {
                "대분류" : '과일', 
                "소분류" : '사과', 
                "작성일" : date, 
                "평점" : grade, 
                "리뷰" : review, 
                "이미지포함" : has_image, 
                "이미지수" : image_count, 
                "도움돼요수" : int(help_count.replace(",", "")), 
                "상품URL" : "https://www.11st.co.kr/products/1102013026"
            }

            if grade == 1: 
                star1.append(result)

            elif grade == 2:
                star2.append(result)

            elif grade == 3:
                star3.append(result)

            elif grade == 4:
                star4.append(result)

            elif grade == 5:
                star5.append(result)
    
        seen_count = len(items)
        print(f"평점 {grade}의 {seen_count}개의 리뷰를 저장 완료")
    
        scroll_attempts = 0
        max_scrolls = 30
        more_button = None
    
        while scroll_attempts < max_scrolls:
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, "button.review-next-list")
                if more_button.is_displayed():
                    print("더보기 버튼이 보입니다.")
                    break
            except:
                pass
    
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.5)
            scroll_attempts += 1
    
        if more_button and more_button.is_displayed():
            more_button.click()
            print("더보기 클릭 완료")
            time.sleep(2)
        else:
            print("더보기 버튼이 보이지 않습니다.")
            break

time.sleep(2)
driver.quit()